# 在windows中安装caffe
关于caffe，不用多说，搞深度学习的都知道。本文结合自己的电脑，记录下windows下caffe的安装过程，以供备忘。  
## 1.编译前的准备工作
在https://github.com/Microsoft/caffe 将项目clone下来，保存在本地，根目录我们后文称之为CAFFE_ROOT。CAFFE_ROOT内有一个和官方Caffe很不同的地方，多了一个windows文件夹。打开文件夹就是微软已经为我们建立好的vs工程，在开始之前需要先将CAFFE_ROOT\windows\CommonSettings.props.example 文件复制一份，并命名为CommonSettings.props 这里保存着编译的一些设置内容。打开CAFFE_ROOT\windows\CommonSettings.props 里面有几点重要的设置需要注意 
```
    <CpuOnlyBuild>false</CpuOnlyBuild>
    <UseCuDNN>true</UseCuDNN>
    <CudaVersion>7.5</CudaVersion>
    <PythonSupport>true</PythonSupport>
    <MatlabSupport>false</MatlabSupport>
```
第一个设置为false表示只使用GPU；第二个选择true表示使用CuDNN，反之为false；第三项为CUDA的版本，使用GPU需要安装CUDA，可以在NVIDIA官网下载[7.5版本的CUDA](https://developer.nvidia.com/cuda-downloads)进行安装；第四项、第五项分别表示是否安装python、matlab支持，这里我选择了安装python支持。  
### 安装cudnn  
在nvdia官网下载对应版本的[cudnn](https://developer.nvidia.com/rdp/cudnn-download)。下载后解压，有三个目录：`bin`、`include`、`lib`。将这三个目录下的文件拷贝到CUDA_HOME对应的目录中，以笔者电脑为例，路径分别是：
* `C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v7.5\bin`
* `C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v7.5\include`
* `C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v7.5\lib\x64`

其实不一定要这三个目录， 但必须是加入PATH的路径。  
用 vs 新建 cuda 项目。在vs编辑器正上方，Solution Configuration 的内容如果是Debug ,改为 Release ,旁边Platforms Solution Platforms 中的内容如果是win32，要改选为x64。

接下来修改项目属性:

项目属性/VC++ Directories/Include Directories 中添加入include的路径(例如 d:\cuda\include);

在项目属性/VC++ Directories/Libary Directories 中添加入lib\x64路径(例如 d:\cuda\lib\x64);

在项目属性/Linker/Input/Additional Dependencies  中添加入cudnn.lib;

项目属性/CUDA C|C++ / Device /Code Generation 中，将sm_20改为 sm_30或更高;

项目属性修改完毕。


加入如下代码：
```C
#include <iostream>
#include <cuda_runtime.h>
#include <cudnn.h>
using namespace std;

void main(){
    cudnnHandle_t handle;
    cudnnStatus_t t = cudnnCreate(&handle);
    cout<< cudnnGetErrorString(t);
    getchar();
}
```


如果结果显示:

`CUDNN_STATES_SUCCESS`

表明安装成功了

### 配置python接口

如果要调用caffe的python接口，需要修改`CommonSettings.props`中的`PythonDir`属性。在文件中找到如下段落：
```xml
    <PropertyGroup Condition="'$(PythonSupport)'=='true'">
        <PythonDir>C:\Miniconda2\</PythonDir>
        <LibraryPath>$(PythonDir)\libs;$(LibraryPath)</LibraryPath>
        <IncludePath>$(PythonDir)\include;$(IncludePath)</IncludePath>
    </PropertyGroup>
```
`PythonDir`属性的默认值是Miniconda2，可以根据需要改成你安装python的根目录，笔者安装python的根目录是`C:\Anaconda2`。因此将配置改为

```xml
    <PropertyGroup Condition="'$(PythonSupport)'=='true'">
        <PythonDir>C:\Anaconda2\</PythonDir>
        <LibraryPath>$(PythonDir)\libs;$(LibraryPath)</LibraryPath>
        <IncludePath>$(PythonDir)\include;$(IncludePath)</IncludePath>
    </PropertyGroup>
```

## 2.编译libcaffe、caffe和pycaffe
打开 CAFFE_ROOT\windows\Caffe.sln 对libcaffe、caffe和pycaffe项目做如下设置  
`项目→属性→C/C++→常规→将警告视为错误 设置为否`
如果不设置的话在编译boost库的时候会由于文字编码的警告而报错  
选择编译环境为Release，x64（其他环境同理）。首先编译libcaffe，在libcaffe上右键生成就可以了。  
接着编译caffe和pycaffe，过程与前者类似。所有编译成功和运行需要的dll文件都会存储在`CAFFE_ROOT\Build\x64\Release`下。至此，Windows版的Caffe编译就成功了。  
最后，需要进行pycaffe的相关配置，将`CAFFE_ROOT\Build\x64\Release\pycaffe`加入环境变量`PythonPath`或者将`CAFFE_ROOT\Build\x64\Release\pycaffe\caffe`拷贝到`<python_root>\Lib\site-packages`下（`<python_root>`为python的安装目录，笔者安装python的根目录是C:\Anaconda2）。

## 3.在python中调用caffe
在导入caffe之前，要事先安装protobuf，安装命令为  
```
pip install protobuf
```  
否则将会报如下错误：
```
ImportError: No module named google.protobuf.internal
```
接着在python中导入caffe
```
import caffe
```
不出意外的话导入应该是成功的

## 4.mnist手写数字分类
接下来用一个mnist手写数字分类的程序来测试一下安装过程是否完整无误，代码来自博客http://www.cnblogs.com/denny402/p/5684431.html


In [1]:
# -*- coding: utf-8 -*-

import caffe
from caffe import layers as L, params as P, proto, to_proto

# 设定文件的保存路径
root = 'G:/workspace/caffe/'  # 根目录
train_list = root + 'mnist/train/train.txt'  # 训练图片列表
test_list = root + 'mnist/test/test.txt'  # 测试图片列表
train_proto = root + 'mnist/train.prototxt'  # 训练配置文件
test_proto = root + 'mnist/test.prototxt'  # 测试配置文件
solver_proto = root + 'mnist/solver.prototxt'  # 参数文件


# 编写一个函数，生成配置文件prototxt
def Lenet(img_list, batch_size, include_acc=False):
    # 第一层，数据输入层，以ImageData格式输入
    data, label = L.ImageData(source=img_list, batch_size=batch_size, ntop=2, root_folder=root,
                              transform_param=dict(scale=0.00390625))
    # 第二层：卷积层
    conv1 = L.Convolution(data, kernel_size=5, stride=1, num_output=20, pad=0, weight_filler=dict(type='xavier'))
    # 池化层
    pool1 = L.Pooling(conv1, pool=P.Pooling.MAX, kernel_size=2, stride=2)
    # 卷积层
    conv2 = L.Convolution(pool1, kernel_size=5, stride=1, num_output=50, pad=0, weight_filler=dict(type='xavier'))
    # 池化层
    pool2 = L.Pooling(conv2, pool=P.Pooling.MAX, kernel_size=2, stride=2)
    # 全连接层
    fc3 = L.InnerProduct(pool2, num_output=500, weight_filler=dict(type='xavier'))
    # 激活函数层
    relu3 = L.ReLU(fc3, in_place=True)
    # 全连接层
    fc4 = L.InnerProduct(relu3, num_output=10, weight_filler=dict(type='xavier'))
    # softmax层
    loss = L.SoftmaxWithLoss(fc4, label)

    if include_acc:  # test阶段需要有accuracy层
        acc = L.Accuracy(fc4, label)
        return to_proto(loss, acc)
    else:
        return to_proto(loss)


def write_net():
    # 写入train.prototxt
    with open(train_proto, 'w') as f:
        f.write(str(Lenet(train_list, batch_size=64)))

    # 写入test.prototxt
    with open(test_proto, 'w') as f:
        f.write(str(Lenet(test_list, batch_size=100, include_acc=True)))


# 编写一个函数，生成参数文件
def gen_solver(solver_file, train_net, test_net):
    s = proto.caffe_pb2.SolverParameter()
    s.train_net = train_net
    s.test_net.append(test_net)
    s.test_interval = 938  # 60000/64，测试间隔参数：训练完一次所有的图片，进行一次测试
    s.test_iter.append(500)  # 50000/100 测试迭代次数，需要迭代500次，才完成一次所有数据的测试
    s.max_iter = 9380  # 10 epochs , 938*10，最大训练次数
    s.base_lr = 0.01  # 基础学习率
    s.momentum = 0.9  # 动量
    s.weight_decay = 5e-4  # 权值衰减项
    s.lr_policy = 'step'  # 学习率变化规则
    s.stepsize = 3000  # 学习率变化频率
    s.gamma = 0.1  # 学习率变化指数
    s.display = 20  # 屏幕显示间隔
    s.snapshot = 938  # 保存caffemodel的间隔
    s.snapshot_prefix = root + 'mnist/lenet'  # caffemodel前缀
    s.type = 'SGD'  # 优化算法
    s.solver_mode = proto.caffe_pb2.SolverParameter.GPU  # 加速
    # 写入solver.prototxt
    with open(solver_file, 'w') as f:
        f.write(str(s))


# 开始训练
def training(solver_proto):
    caffe.set_device(0)
    caffe.set_mode_gpu()
    solver = caffe.SGDSolver(solver_proto)
    solver.solve()


#
if __name__ == '__main__':
    write_net()
    gen_solver(solver_proto, train_proto, test_proto)
    training(solver_proto)

运行日志